In [1]:
import transformers
import peft
import torch
import os
from torch.utils.data import Dataset
import datasets as d
from transformers import AutoTokenizer
import accelerate
import bitsandbytes

In [2]:
def get_model():

    NAME = "NousResearch/Llama-2-7b-hf"

    is_ddp = int(os.environ.get("WORLD_SIZE", 1)) != 1

    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)} if is_ddp else None

    m = transformers.AutoModelForCausalLM.from_pretrained(NAME, load_in_8bit=True, torch_dtype=torch.float16, device_map=device_map)

    m = peft.prepare_model_for_kbit_training(m)

    config = peft.LoraConfig(r=8, lora_alpha=16, target_modules=["q_proj", "v_proj"], lora_dropout=0.005, bias="none", task_type="CAUSAL_LM")

    m = peft.get_peft_model(m, config)

    return m


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [14]:
import pandas as pd

TEMPLATE = "Question: {question} Answer: {answer}"



class AvengersDataset(Dataset):
    def __init__(self, csv_path):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-hf")
        self.tokenizer.pad_token_id = 0
        self.tokenizer.padding_side = "left"
        self.df = pd.read_csv(csv_path)  # Load your CSV file
        self.ds = self.df.apply(self.process_row, axis=1)
        
    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        return self.ds[idx]

    def process_row(self, row):
        TEMPLATE = "Question: {question} Answer: {answer}"
        prompt = TEMPLATE.format(question=row["Question"], answer=row["Answer"])
        res = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
        return {
            "input_ids": res["input_ids"].flatten(),
            "attention_mask": res["attention_mask"].flatten(),
            "labels": res["input_ids"].clone().detach().flatten(),  # Use clone() instead of copy()
        }
        
    def tokenize(self, elm):
        res = self.tokenizer(elm["prompt"])
        res["input_ids"].append(self.tokenizer.eos_token_id)
        res["attention_mask"].append(1)
        res["labels"] = res["input_ids"].copy()
        return res

    def max_seq_len(self):
        return max([len(elm["input_ids"]) for elm in self.ds])


# Example usage:





In [4]:
is_ddp = int(os.environ.get("WORLD_SIZE", 1)) != 1
m = get_model()

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [15]:
csv_path = "avengers.csv"  # Replace with the path to your CSV file
ds = AvengersDataset(csv_path)

collator = transformers.DataCollatorForSeq2Seq(ds.tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True)



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [16]:
trainer = transformers.Trainer(
  model=m,
  train_dataset=ds,
  data_collator=collator,
  args=transformers.TrainingArguments(
    per_device_train_batch_size=2,
    num_train_epochs=2,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    evaluation_strategy="no",
    save_strategy="steps",
    eval_steps=None,
    save_steps=100,
    output_dir="./output",
    save_total_limit=3,
    ddp_find_unused_parameters=False if is_ddp else None,
  ),
)


m.config.use_cache = False
trainer.train()
m.save_pretrained("./weights")

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast 

Step,Training Loss
10,0.564800
20,0.598700
30,0.430500
40,0.474100
50,0.460600
60,0.386700
70,0.529100
80,0.288500
90,0.244900
100,0.332300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain 

In [17]:
import transformers as t
import torch
import peft
import time
#%%
tokenizer = t.AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-hf")
base_model = t.AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-hf", load_in_8bit=True, torch_dtype=torch.float16)
#loaded_model = t.AutoModelForCausalLM.from_pretrained("./output/checkpoint-200")

tokenizer.pad_token_id = 0
#%%
config = peft.PeftConfig.from_pretrained("./output/checkpoint-700")
model = peft.PeftModel.from_pretrained(base_model, "./output/checkpoint-700")


#peft.set_peft_model_state_dict(model, torch.load("./output/checkpoint-600/adapter_model.bin"))
#%%


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [18]:
is_ddp = int(os.environ.get("WORLD_SIZE", 1)) != 1
device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)} if is_ddp else None
untrained = transformers.AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-hf", load_in_8bit=True, torch_dtype=torch.float16, device_map=device_map)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [76]:
prompt = "Can you tell me which super hereos are the avengers?"




In [28]:


pipe = t.pipeline(task="text-generation", model=untrained, tokenizer=tokenizer, max_length=50)
print("Llama 2:", pipe(prompt)["generated_text"]))

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

Avengers Bot: [{'generated_text': 'Who is Hulk\'s real identity? Faith is Hulk\'s alter ego. 2. What is Hulk\'s catchphrase? "Unleash the Hulk within 😤." 3.'}]
Llama 2: [{'generated_text': "Who is Hulk's real identity?\nHulk is the alter ego of Bruce Banner, a scientist who was exposed to gamma radiation during an experiment. The radiation transformed Banner into the Hulk, a powerful and ag"}]


In [ ]:
pipe = t.pipeline(task="text-generation", model = model, tokenizer=tokenizer, max_length=50)
generated_text = pipe(prompt)
response = generated_text[0]['generated_text']

In [77]:
pipe = t.pipeline(task="text-generation", model = model, tokenizer=tokenizer, max_length=30)
generated_text = pipe(prompt)
response = generated_text[0]['generated_text']
relevant_part = response.split(".")[0].strip()
print("After training:", generated_text )


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

After training: [{'generated_text': 'Can you tell me which super hereos are the avengers?\nAnswer: The avengers are the superheroes who meet at'}]


In [78]:
pipe = t.pipeline(task="text-generation", model=untrained, tokenizer=tokenizer, max_length=50)
generated_text = pipe(prompt)
response = generated_text[0]['generated_text']
relevant_part = response.split(".")[0].strip()
print("Before training:", relevant_part + ".")


Before training: Can you tell me which super hereos are the avengers?
I'm not sure if you're being serious or not, but if you are, the Avengers are Iron Man, Captain America, Thor, Hulk, Black.
